# Imports

In [ ]:
import torchvision
import torchvision.transforms as transforms
import torch
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split, Subset, WeightedRandomSampler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import math
import torch.nn as nn
from torch.utils.data import TensorDataset
from sklearn.metrics import accuracy_score
random.seed(7)

# Data preparation

In [ ]:
# train and test set
train_set=torchvision.datasets.FashionMNIST(root= './MNISTT_data/',
                                  download= True,
                                  transform= transforms.Compose([transforms.ToTensor()]))
test_set=torchvision.datasets.FashionMNIST(root= './MNISTT_data/',
                                  train=False,
                                  download= True,
                                  transform= transforms.Compose([transforms.ToTensor()]))


Extracting ./MNISTT_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./MNISTT_data/FashionMNIST/raw


Extracting ./MNISTT_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./MNISTT_data/FashionMNIST/raw


Extracting ./MNISTT_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./MNISTT_data/FashionMNIST/raw




Extracting ./MNISTT_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNISTT_data/FashionMNIST/raw
Processing...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [ ]:
# split to train and validation
targets=torch.tensor(train_set.targets)
targets = targets[torch.randperm(len(targets))]

x_train = torch.stack([x[0] for x in train_set], dim=0)
y_train = torch.FloatTensor([x[1] for x in train_set])


x_test = torch.stack([x[0] for x in test_set], dim=0)
y_test = torch.FloatTensor([x[1] for x in test_set])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


# Analyze distributions

In [ ]:
 output_mapping = {
                 0: "T-shirt/Top",
                 1: "Trouser",
                 2: "Pullover",
                 3: "Dress",
                 4: "Coat", 
                 5: "Sandal", 
                 6: "Shirt",
                 7: "Sneaker",
                 8: "Bag",
                 9: "Ankle Boot"
                 }

In [ ]:
train_counts=y_train.unique(return_counts=True)
test_counts=y_test.unique(return_counts=True)

for i in range(10):
    t=(100*train_counts[1][i]/(train_counts[1][i] + test_counts[1][i])).item()
    v=(100*test_counts[1][i]/(train_counts[1][i] + test_counts[1][i])).item()
    print( (output_mapping[i] + ": Train= {t:.2f}%;  Test= {v:.2f}%").format(t=t, v=v))
    

T-shirt/Top: Train= 85.71%;  Test= 14.29%
Trouser: Train= 85.71%;  Test= 14.29%
Pullover: Train= 85.71%;  Test= 14.29%
Dress: Train= 85.71%;  Test= 14.29%
Coat: Train= 85.71%;  Test= 14.29%
Sandal: Train= 85.71%;  Test= 14.29%
Shirt: Train= 85.71%;  Test= 14.29%
Sneaker: Train= 85.71%;  Test= 14.29%
Bag: Train= 85.71%;  Test= 14.29%
Ankle Boot: Train= 85.71%;  Test= 14.29%


# Prepare data for federated learning

In [ ]:
def split_and_shuffle_labels(y_data, seed, amount):
    y_data=pd.DataFrame(y_data,columns=["labels"])
    y_data["i"]=np.arange(len(y_data))
    label_dict = dict()
    for i in range(10):
        var_name=  output_mapping[i]
        label_info=y_data[y_data["labels"]==i]
        np.random.seed(seed)
        label_info=np.random.permutation(label_info)
        label_info=label_info[0:amount]
        label_info=pd.DataFrame(label_info, columns=["labels","i"])
        label_dict.update({var_name: label_info })
    return label_dict

This function splis the data with respect to the labels to 10 dataframes (each dataframe contains one unique label and the indices of the different pîctures annotated with that label).

They are stored in a dictionnary where the key is the label and the value is the considered dataframe.

In [ ]:
#example: Trouser class(label 1)
split_and_shuffle_labels(y_train, 7, 6000)["Trouser"]

,labels,i
0,1.0,33366.0
1,1.0,54556.0
2,1.0,23364.0
3,1.0,43293.0
4,1.0,32819.0
...,...,...
5995,1.0,56938.0
5996,1.0,25205.0
5997,1.0,4872.0
5998,1.0,12002.0


In [ ]:
def get_iid_subsamples_indices(label_dict, number_of_samples, amount):
    sample_dict= dict()
    batch_size=int(math.floor(amount/number_of_samples))
    for i in range(number_of_samples):
        sample_name="sample"+str(i)
        dumb=pd.DataFrame()
        for j in range(10):
            label_name=output_mapping[j]
            a=label_dict[label_name][i*batch_size:(i+1)*batch_size]
            dumb=pd.concat([dumb,a], axis=0)
        dumb.reset_index(drop=True, inplace=True)    
        sample_dict.update({sample_name: dumb}) 
    return sample_dict

This function returns a dictionnary of 100 keys (100 devices), each key is given by "sample+i" and contains the same number of samples equally distributed to all the devices, except for the last device which takes the remaining pictures.

In [ ]:
label_dict_train=split_and_shuffle_labels(y_data=y_train, seed=1, amount=4500)
sample_dict_train=get_iid_subsamples_indices(label_dict=label_dict_train, number_of_samples=100, amount=4500)


label_dict_test=split_and_shuffle_labels(y_data=y_test, seed=1, amount=900)
sample_dict_test=get_iid_subsamples_indices(label_dict=label_dict_test, number_of_samples=100, amount=900)

In [ ]:
#example: train first sample distrubution
sample_dict_train["sample0"]["labels"].value_counts()

9.0    45
8.0    45
7.0    45
6.0    45
5.0    45
4.0    45
3.0    45
2.0    45
1.0    45
0.0    45
Name: labels, dtype: int64

In [ ]:
def create_iid_subsamples(sample_dict, x_data, y_data, x_name, y_name):
    x_data_dict= dict()
    y_data_dict= dict()
    
    for i in range(len(sample_dict)): 
        xname= x_name+str(i)
        yname= y_name+str(i)
        sample_name="sample"+str(i)
        
        indices=np.sort(np.array(sample_dict[sample_name]["i"]))
        
        x_info= x_data[indices,:]
        x_data_dict.update({xname : x_info})
        
        y_info= y_data[indices]
        y_data_dict.update({yname : y_info})
        
    return x_data_dict, y_data_dict

This functions extracts x_train, y_train, x_test and y_test dictionnaries from the latest obtained dictionnary. Each element from the dictionnary is specific to a device. 

In [ ]:
x_train_dict, y_train_dict = create_iid_subsamples(sample_dict=sample_dict_train, x_data=x_train, y_data=y_train, x_name="x_train", y_name="y_train")
x_test_dict, y_test_dict = create_iid_subsamples(sample_dict=sample_dict_test, x_data=x_test, y_data=y_test, x_name="x_test", y_name="y_test")

In [ ]:
#example shape of first element of x_train 
x_train_dict["x_train0"].shape

torch.Size([450, 1, 28, 28])

In [ ]:
#example shape of first element of y_test
y_test_dict["y_test0"].shape

torch.Size([90])

In [ ]:
def compare_local_and_merged_model_performance(number_of_samples):
    accuracy_table=pd.DataFrame(data=np.zeros((number_of_samples,3)), columns=["sample", "local_ind_model", "merged_main_model"])
    for i in range (number_of_samples):
    
        test_ds = TensorDataset(x_test_dict[name_of_x_test_sets[i]], y_test_dict[name_of_y_test_sets[i]])
        test_dl = DataLoader(test_ds, batch_size=batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        individual_loss, individual_accuracy = validation(model, test_dl, criterion)
        main_loss, main_accuracy =validation(main_model, test_dl, main_criterion )
    
        accuracy_table.loc[i, "sample"]="sample "+str(i)
        accuracy_table.loc[i, "local_ind_model"] = individual_accuracy
        accuracy_table.loc[i, "merged_main_model"] = main_accuracy

    return accuracy_table

# Model

In [ ]:
class cnn(nn.Module):
    def __init__(self):
        super(cnn, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        print(out.shape)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        x = self.fc3(out)
        
        return x

In [ ]:
def create_model_optimizer_criterion_dict(number_of_samples):
    model_dict = dict()
    optimizer_dict= dict()
    criterion_dict = dict()
    
    for i in range(number_of_samples):
        model_name="model"+str(i)
        model_info=cnn()
        model_dict.update({model_name : model_info })
        
        optimizer_name="optimizer"+str(i)
        optimizer_info = torch.optim.SGD(model_info.parameters(), lr=learning_rate, momentum=momentum)
        optimizer_dict.update({optimizer_name : optimizer_info })
        
        criterion_name = "criterion"+str(i)
        criterion_info = nn.CrossEntropyLoss()
        criterion_dict.update({criterion_name : criterion_info})
        
    return model_dict, optimizer_dict, criterion_dict 

Similarly to the datasets dictionnaries, we create a local model with respect to each device and we assign each model to its respective key. 

In [ ]:
# parameters for train
number_of_samples=100
learning_rate = 0.01
numEpoch = 10
batch_size = 32
momentum = 0.9

In [ ]:
def get_averaged_weights(model_dict, number_of_samples):
   
    fc1_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc1.weight.shape)
    fc1_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc1.bias.shape)
    
    
    fc2_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc2.weight.shape)
    fc2_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc2.bias.shape)
    
    fc3_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc3.weight.shape)
    fc3_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc3.bias.shape)
    
    conv2d1_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].layer1[0].weight.shape)
    conv2d1_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].layer1[0].bias.shape)

    conv2d2_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].layer2[0].weight.shape)
    conv2d2_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].layer2[0].bias.shape)
    
    with torch.no_grad():
    
    
        for i in range(number_of_samples):
            fc1_mean_weight += model_dict[name_of_models[i]].fc1.weight.data.clone()
            fc1_mean_bias += model_dict[name_of_models[i]].fc1.bias.data.clone()
        
            fc2_mean_weight += model_dict[name_of_models[i]].fc2.weight.data.clone()
            fc2_mean_bias += model_dict[name_of_models[i]].fc2.bias.data.clone()
        
            fc3_mean_weight += model_dict[name_of_models[i]].fc3.weight.data.clone()
            fc3_mean_bias += model_dict[name_of_models[i]].fc3.bias.data.clone()
            
            conv2d1_mean_weight+=model_dict[name_of_models[0]].layer1[0].weight.data.clone()
            conv2d1_mean_bias+=model_dict[name_of_models[0]].layer1[0].bias.data.clone()

            conv2d2_mean_weight+=model_dict[name_of_models[0]].layer2[0].weight.data.clone()
            conv2d2_mean_bias+=model_dict[name_of_models[0]].layer2[0].bias.data.clone()
        
        fc1_mean_weight =fc1_mean_weight/number_of_samples
        fc1_mean_bias = fc1_mean_bias/ number_of_samples
    
        fc2_mean_weight =fc2_mean_weight/number_of_samples
        fc2_mean_bias = fc2_mean_bias/ number_of_samples
    
        fc3_mean_weight =fc3_mean_weight/number_of_samples
        fc3_mean_bias = fc3_mean_bias/ number_of_samples
        
        conv2d1_mean_weight =conv2d1_mean_weight/number_of_samples
        conv2d1_mean_bias = conv2d1_mean_bias/ number_of_samples
        
        conv2d2_mean_weight =conv2d2_mean_weight/number_of_samples
        conv2d2_mean_bias = conv2d2_mean_bias/ number_of_samples
   
    return fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias, conv2d1_mean_weight, conv2d1_mean_bias, conv2d2_mean_weight, conv2d2_mean_bias


This function takes the average of weights & biases in individual nodes.
We started by initializing an instant for both weights and biases of each layer, then fill it with the updated weights and biases of each node (i.e sample).
Finally, we calculate the average  weight and biases from all the collected parameters of all samples. 


In [ ]:
def set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_samples):
    fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias, conv2d1_mean_weight, conv2d1_mean_bias, conv2d2_mean_weight, conv2d2_mean_bias = get_averaged_weights(model_dict, number_of_samples=number_of_samples)
    
    with torch.no_grad():
        main_model.fc1.weight.data = fc1_mean_weight.data.clone()
        main_model.fc2.weight.data = fc2_mean_weight.data.clone()
        main_model.fc3.weight.data = fc3_mean_weight.data.clone()
        main_model.layer1[0].weight.data = conv2d1_mean_weight.data.clone()
        main_model.layer2[0].weight.data = conv2d2_mean_weight.data.clone()

        main_model.fc1.bias.data = fc1_mean_bias.data.clone()
        main_model.fc2.bias.data = fc2_mean_bias.data.clone()
        main_model.fc3.bias.data = fc3_mean_bias.data.clone() 
        main_model.layer1[0].bias.data = conv2d1_mean_bias.data.clone()
        main_model.layer2[0].bias.data = conv2d2_mean_bias.data.clone()

    return main_model

This function sends the averaged weights of individual local devices to the main model and sets them as the new weights of the main model.

In [ ]:
def send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_samples):
    with torch.no_grad():
        for i in range(number_of_samples):

            model_dict[name_of_models[i]].fc1.weight.data =main_model.fc1.weight.data.clone()
            model_dict[name_of_models[i]].fc2.weight.data =main_model.fc2.weight.data.clone()
            model_dict[name_of_models[i]].fc3.weight.data =main_model.fc3.weight.data.clone()
            model_dict[name_of_models[i]].layer1[0].weight.data =main_model.layer1[0].weight.data.clone()
            model_dict[name_of_models[i]].layer2[0].weight.data =main_model.layer2[0].weight.data.clone()
            
            model_dict[name_of_models[i]].fc1.bias.data =main_model.fc1.bias.data.clone()
            model_dict[name_of_models[i]].fc2.bias.data =main_model.fc2.bias.data.clone()
            model_dict[name_of_models[i]].fc3.bias.data =main_model.fc3.bias.data.clone() 
            model_dict[name_of_models[i]].layer1[0].bias.data =main_model.layer1[0].bias.data.clone()
            model_dict[name_of_models[i]].layer2[0].bias.data =main_model.layer2[0].bias.data.clone()
    
    return model_dict

This function sends the parameters of the main model to the nodes. It returns an updated version of  model_dict.


In [ ]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct = 0

    for data, target in train_loader:
        output = model(data).cuda()
        loss = criterion(output, target.cuda().type( torch.int64))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        prediction = output.argmax(dim=1, keepdim=True).cuda()
        correct += prediction.cpu().eq(target.cpu().view_as(prediction.cpu())).sum().item()
        

    return train_loss / len(train_loader), correct/len(train_loader.dataset)

This function is used to train a model on its data (train_loader), with a specified criterion and optimizer. 

In [ ]:
def validation(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data).cuda()
            test_loss += criterion(output, target.cuda().type( torch.int64) ).item()
            prediction = output.argmax(dim=1, keepdim=True).cuda()
            correct += prediction.cpu().eq(target.cpu().view_as(prediction.cpu())).sum().item()

    test_loss /= len(test_loader)
    correct /= len(test_loader.dataset)

    return (test_loss, correct)


This function evaluates the model on the test data

In [ ]:
def start_train_end_node_process(number_of_samples):
    for i in range (number_of_samples): 

        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        test_ds = TensorDataset(x_test_dict[name_of_x_test_sets[i]], y_test_dict[name_of_y_test_sets[i]])
        test_dl = DataLoader(test_ds, batch_size=batch_size * 2)
        
        
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        print("Subset" ,i)
        for epoch in range(numEpoch):        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            val_loss, val_accuracy = validation(model, test_dl, criterion)
    
            print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.5f}".format(train_accuracy) + " | validation accuracy: {:7.5f}".format(val_accuracy))

This function does the train of a specific node (a local device) for 100 epochs on its own mini-dataset.

In [ ]:
def start_train_end_node_process_without_print(number_of_samples):
    for i in range (number_of_samples): 

        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        test_ds = TensorDataset(x_test_dict[name_of_x_test_sets[i]], y_test_dict[name_of_y_test_sets[i]])
        test_dl = DataLoader(test_ds, batch_size=batch_size * 2)
        
        
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        for epoch in range(numEpoch):        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            val_loss, val_accuracy = validation(model, test_dl, criterion)


This is the same previous function except that we don't print the process in each node.

In [ ]:
main_model = cnn()
main_optimizer = torch.optim.SGD(main_model.parameters(), lr=0.01, momentum=0.9)
main_criterion = nn.CrossEntropyLoss()

The created model is a Convolutional network. We create an instance from the defined cnn class along with the SGD optimizer and CrossEntropyLoss to track the error.

In [ ]:
model_dict, optimizer_dict, criterion_dict = create_model_optimizer_criterion_dict(number_of_samples)
#fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias, conv2d1_mean_weight, conv2d1_mean_bias, conv2d2_mean_weight, conv2d2_mean_bias = get_averaged_weights(model_dict, number_of_samples)

In [ ]:
#List of the names of data partitions
name_of_x_train_sets=list(x_train_dict.keys())
name_of_y_train_sets=list(y_train_dict.keys())
name_of_x_test_sets=list(x_test_dict.keys())
name_of_y_test_sets=list(y_test_dict.keys())

#List of the names of models, optimizers and criterions
name_of_models=list(model_dict.keys())
name_of_optimizers=list(optimizer_dict.keys())
name_of_criterions=list(criterion_dict.keys())


In [ ]:
print(main_model.fc2.weight[0:1,0:5])
print(model_dict["model1"].fc2.weight[0:1,0:5])

tensor([[-0.0360, -0.0361,  0.0203, -0.0196, -0.0389]],
       grad_fn=<SliceBackward>)
tensor([[ 0.0095,  0.0150, -0.0198,  0.0215,  0.0015]],
       grad_fn=<SliceBackward>)


In [ ]:
model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_samples)

In [ ]:
print(main_model.fc2.weight[0:1,0:5])
print(model_dict["model1"].fc2.weight[0:1,0:5])

tensor([[-0.0360, -0.0361,  0.0203, -0.0196, -0.0389]],
       grad_fn=<SliceBackward>)
tensor([[-0.0360, -0.0361,  0.0203, -0.0196, -0.0389]],
       grad_fn=<SliceBackward>)


We can see that the updates from the main model have been made successfully.

In [ ]:
start_train_end_node_process(number_of_samples)

This training is only done for 1 iteration.
We can see that the accuracies are not that great.

It's normal because we haven't sent the local trained weights to the main model. Once we send the weights to the main model, they're averaged and sent back to the local nodes. Then, this process is repeated until we achieve 10 iterations.

In [ ]:
test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

In [ ]:
before_acc_table=compare_local_and_merged_model_performance(number_of_samples=number_of_samples)
before_test_loss, before_test_accuracy = validation(main_model, test_dl, main_criterion)
#averaging for main model
main_model= set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_samples) 
after_acc_table=compare_local_and_merged_model_performance(number_of_samples=number_of_samples)
after_test_loss, after_test_accuracy = validation(main_model, test_dl, main_criterion)

In [ ]:
print("Federated main model vs individual local models before FedAvg first iteration")
before_acc_table.head()

Federated main model vs individual local models before FedAvg first iteration


,sample,local_ind_model,merged_main_model
0,sample 0,0.755556,0.100000
1,sample 1,0.722222,0.111111
2,sample 2,0.711111,0.100000
3,sample 3,0.711111,0.077778
4,sample 4,0.811111,0.133333


After training local models for 1 iteration and after sending the weights and before averaging them by the main model we can see that the main model has very bad accuracy because it was randomly initialiazed. 
\
We must send the weights of the local nodes to the main i order to average them and get more reasonable accuracy.

In [ ]:
print("Federated main model vs individual local models after FedAvg first iteration")
after_acc_table.head()

Federated main model vs individual local models after FedAvg first iteration


,sample,local_ind_model,merged_main_model
0,sample 0,0.755556,0.444444
1,sample 1,0.722222,0.455556
2,sample 2,0.711111,0.366667
3,sample 3,0.711111,0.411111
4,sample 4,0.811111,0.455556


After training local models for 1 iteration and after sending the weights and after averaging them by the main model we can see that the main model still has medium accuracy. 
\
We must train for more iterations in local nodes and repeat the same process.

In [ ]:
for i in range(10):
    #send weights to local nodes
    model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_samples)
    #train local nodes
    start_train_end_node_process_without_print(number_of_samples)
    
    main_model= set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_samples) 
    test_loss, test_accuracy = validation(main_model, test_dl, main_criterion)
    print("Iteration", str(i+2), ": main_model accuracy on all test data: {:7.4f}".format(test_accuracy))

Iteration 2 : main_model accuracy on all test data:  0.5113
Iteration 3 : main_model accuracy on all test data:  0.3891
Iteration 4 : main_model accuracy on all test data:  0.2513
Iteration 5 : main_model accuracy on all test data:  0.2240
Iteration 6 : main_model accuracy on all test data:  0.2018
Iteration 7 : main_model accuracy on all test data:  0.2079
Iteration 8 : main_model accuracy on all test data:  0.1988
Iteration 9 : main_model accuracy on all test data:  0.2214
Iteration 10 : main_model accuracy on all test data:  0.2777
Iteration 11 : main_model accuracy on all test data:  0.2556


The same iterations are repeated 10 times and the accuracy improves gradually as the main model learns more from the local nodes each iteration.
\
It is not a perfect result and may be less performant than the normal training on ine unique device; but the results are quite intresting.